## Hit 150 stores per zip code in california 

In [1]:
import pandas as pd

In [2]:
affluence = pd.read_csv('./datasets/16zpallagi.csv')
affluence.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
0,1,AL,0,1,815440,477700,105350,221200,440830,1296920,...,367320,330066,0,0,0,0,63420,51444,711580,1831661
1,1,AL,0,2,495830,211930,142340,128890,272440,996240,...,398050,984297,0,0,0,0,74090,110889,416090,1173463
2,1,AL,0,3,263390,83420,137870,36340,154880,584000,...,253180,1349246,0,0,0,0,64000,143060,195130,543284
3,1,AL,0,4,167190,29420,124060,10610,99700,421720,...,165830,1425430,0,0,0,0,45020,128920,117410,381329
4,1,AL,0,5,217440,20240,188080,4880,129410,601040,...,216720,3922449,390,155,60,19,82940,423629,126130,506526


In [3]:
affluence.shape

(179796, 147)

In [4]:
affluence['zipcode'].nunique()

29874

In [5]:
#according to the documentation, we must look at agi stub

In [6]:
features = ['agi_stub', 'zipcode']
df = affluence[features]

In [7]:
mother_df = df.groupby('zipcode', as_index=False)['agi_stub'].mean()

In [8]:
mother_df.head()

,zipcode,agi_stub
0,0,3.5
1,1001,3.5
2,1002,3.5
3,1003,3.4
4,1005,3.5


In [9]:
mother_df.shape

(29874, 2)

In [10]:
mother_df.dtypes

zipcode       int64
agi_stub    float64
dtype: object

In [11]:
mother_df['zipcode']=mother_df['zipcode'].apply(lambda x: '{0:0>5}'.format(x))
#shoutout to stack overflow- this code adds a zero in front of the zipcode if there are digits less than 5

In [12]:
mother_df.head()

,zipcode,agi_stub
0,00000,3.5
1,01001,3.5
2,01002,3.5
3,01003,3.4
4,01005,3.5


In [13]:
mother_df['zipcode'][27097]

'90001'

In [14]:
mother_df = mother_df.iloc[27097:] #dropping first row 

In [15]:
mother_df.head(20)

,zipcode,agi_stub
27097,90001,3.5
27098,90002,3.5
27099,90003,3.5
27100,90004,3.5
27101,90005,3.5
27102,90006,3.5
27103,90007,3.5
27104,90008,3.5
27105,90010,3.5
27106,90011,3.5


In [16]:
zipcodes = mother_df['zipcode']

In [17]:
type(zipcodes)

pandas.core.series.Series

In [18]:
import requests
import time

In [19]:
list_of_keys = []
#due to confidentiality, we are removing our keys
#since we have a total of three people in our group, we created a list of keys in order to scrape more data at once 

In [20]:
import time

In [2]:
mother = []
ENDPOINT = 'https://api.yelp.com/v3/businesses/search'

i = 0
key = list_of_keys[i]
for codes in zipcodes:
    for offsetnum in range(0,150, 50):
        HEADERS = {'Authorization': 'bearer %s' % key}
        PARAMETERS = {'location': codes,
                        'limit': 50,
                        'offset': offsetnum

                            }  
        response = requests.get(url = ENDPOINT, params= PARAMETERS, headers = HEADERS)
        #print(response.status_code)
        if response.status_code == 429:
            i = i + 1
            key = list_of_keys[i]
            HEADERS = {'Authorization': 'bearer %s' % key}
            response = requests.get(url = ENDPOINT, params= PARAMETERS, headers = HEADERS)
            business_data = response.json()
            mother.extend(business_data['businesses'])
            print(f'hit {codes} using {key}')
#         elif response.status_code == 400:
#             break
        elif response.status_code == 500:
            break
        else:
            business_data = response.json()
            mother.extend(business_data['businesses'])
            print(f'hit {codes} using {key}')
                


            #if statement where all businesses in that city has been fulfilled then break 
                
        time.sleep(.3)
        
        

NameError: name 'list_of_keys' is not defined

In [22]:
import pandas as pd

In [23]:
# initialize list of dicts
post_info = []
# loop through dicts and get specific info
for b in mother:
    
    info = {}
    info['id'] = b['id']
    info['latitude'] = b['coordinates']['latitude']
    info['longitude'] = b['coordinates']['longitude']
    info['rating'] = b['rating']
    info['address'] = b['location']['address1']
    info['city'] = b['location']['city']
    info['zip_code'] = b.get('location').get('zip_code', None)
    info['phone'] = b['phone']
    info['name'] = b['name']
    info['price'] = b.get('price', None)
    #info['category1'] = b['categories'][0].get('alias', None)
    try:
        info['category1'] = b['categories'][0].get('alias', None)
        info['category2'] = b['categories'][1].get('alias', None)
        info['category3'] = b['categories'][2].get('alias', None)
        info['category4'] = b['categories'][3].get('alias', None)
        info['category5'] = b['categories'][4].get('alias', None)
    except: None
    post_info.append(info)   
# create dataframe    
post_info_df = pd.DataFrame(post_info)

In [24]:
post_info_df.shape

(277159, 15)

In [25]:
post_info_df.drop_duplicates(subset = 'id', keep= 'last', inplace = True)

In [26]:
post_info_df.shape

(85011, 15)

In [27]:
post_info_df[post_info_df['id'].duplicated()]

,address,category1,category2,category3,category4,category5,city,id,latitude,longitude,name,phone,price,rating,zip_code


In [28]:
post_info_df.head()

,address,category1,category2,category3,category4,category5,city,id,latitude,longitude,name,phone,price,rating,zip_code
71,1517 E Florence Ave,mexican,NaN,NaN,NaN,NaN,Los Angeles,RVJ7CtzHbSIXySMJ7uJw9w,33.974990,-118.246960,El Senor Taco,+13235824709,$,3.5,90001
109,7600 Graham Ave,parks,playgrounds,recreation,NaN,NaN,Los Angeles,73_9SM3HCAPZi6VmjjY2Rg,33.970200,-118.242040,Franklin D Roosevelt Park,+13235867201,None,4.5,90001
111,None,desserts,chocolate,NaN,NaN,NaN,Los Angeles,XuDsBBkKMgyb19qrCG-Vxw,33.973630,-118.249890,Brigadier Sweets,+12132811821,$$,5.0,90001
118,None,foodtrucks,african,NaN,NaN,NaN,Los Angeles,E02jtyN7b9LJW8bJwX97kA,33.978530,-118.249700,African Chop,+14084295458,None,4.5,90001
129,757 S La Brea Ave,foodtrucks,mexican,NaN,NaN,NaN,Los Angeles,KWKkQHHwVBPS_4abj-DaYw,34.060716,-118.344931,Huitlacoche,+13233810688,$,4.5,90017


In [29]:
post_info_df.to_csv('./datasets/CA')

In [33]:
post_info_df[post_info_df['name'] == 'Ruen Pair']

,address,category1,category2,category3,category4,category5,city,id,latitude,longitude,name,phone,price,rating,zip_code
8762,5257 Hollywood Blvd,thai,NaN,NaN,NaN,NaN,Los Angeles,UdpE_5k2c3zsGBFdcAjMGA,34.101885,-118.30513,Ruen Pair,+13234660153,$$,4.0,90027


In [38]:
post_info_df['zip_code'].nunique()

3444

In [39]:
post_info_df['zip_code'].value_counts()

           288
96720      235
92101      203
96740      184
96150      171
92262      162
94110      160
95240      155
98225      153
96161      152
93230      151
98499      149
95020      148
98801      147
93446      147
95603      145
99801      144
96753      144
94513      144
95928      144
91355      143
92054      143
92243      142
96761      142
95060      142
93257      141
95076      140
95630      140
95376      140
95037      139
          ... 
V2R 4E1      1
V2R 4H2      1
94595        1
V2P 4N8      1
V2S 4A5      1
28075        1
77031        1
22116        1
46032        1
V0H 1B0      1
44401        1
V2P 2L7      1
97461        1
V0S 1N0      1
98907        1
95637        1
V2P 7K6      1
V2T 6K2      1
V3S 9N8      1
91008        1
99707        1
91399        1
93932        1
V2S 5A1      1
V3S 9S4      1
93201        1
98504        1
93450        1
93276        1
98643        1
Name: zip_code, Length: 3444, dtype: int64